In [2]:
import os
import sys
app_path = os.path.abspath('..')
sys.path.insert(0, app_path)

from dotenv import load_dotenv

from app.telegram_client import get_client_with_auth
from app.chroma_client import get_embeddings, get_client, load_data_to_chroma


load_dotenv()

CHANNEL_NAME = os.getenv('TELEGRAM_CHAT_NAME')
CHANNEL_ID = os.getenv('TELEGRAM_CHAT_ID')
try:
    CHANNEL_ID = int(CHANNEL_ID)
except:
    pass

print(f'Channel name is {CHANNEL_NAME}')
print(f'Channel id is {CHANNEL_ID}')

Channel name is go_to_vilnius_e5
Channel id is -1133953167


In [3]:
telegram_client = await get_client_with_auth()

Server sent a very old message with ID 7500594990775182337, ignoring (see FAQ for details)
Server sent a very old message with ID 7500595128046645249, ignoring (see FAQ for details)


In [4]:
from typing import Any
from tqdm import tqdm
from telethon.tl.types import PeerChannel

CHAT_MESSAGES_LIMIT = 1000

async def get_messages_from_channel(
    telegram_client, channel_id: int | str, min_id: int = 0, max_id: int = 0, chunks: int = 10
) -> dict[int, dict[str, Any]]:
    messages_data = {}
    if channel_id < 0:
        channel_id = PeerChannel(channel_id)

    channel = await telegram_client.get_entity(channel_id)
    last_seen_message_id = max_id

    for _i in tqdm(range(chunks), desc="Fetching chunks", position=0, leave=False, colour='green'):
        if all((last_seen_message_id, min_id)) and last_seen_message_id == min_id:
            print("Reached last unseen message")
            break

        async for message in telegram_client.iter_messages(
            channel, min_id=min_id, offset_id=last_seen_message_id, limit=CHAT_MESSAGES_LIMIT
        ):
            if not message.text or len(message.text) < 10:
                continue

            message_text = message.text
            if message.is_reply:
                if message.reply_to_msg_id not in messages_data:
                    original_message = await message.get_reply_message()
                    if original_message:
                        messages_data[original_message.id] = {
                            'text': original_message.text,
                            'metadata': {
                                'sender_name': original_message.sender.username or '',
                                'id': original_message.id,
                                'date_str': original_message.date.isoformat(),
                                'date': original_message.date.timestamp(),
                            }
                        }

                original_message = messages_data.get(message.reply_to_msg_id)
                if original_message:
                    message_text = f'>> {original_message['text']}\n\n {message.text}'
                else:
                    message_text = f'>> [[ORIGINAL MESSAGE REMOVED]]\n\n {message.text}'

            messages_data[message.id] = {
                'text': message_text,
                'metadata': {
                    'sender_name': message.sender.username or '',
                    'id': message.id,
                    'date': message.date.timestamp(),
                }
            }

        last_seen_message_id = message.id
        if last_seen_message_id == 0:
            break

    return messages_data

In [5]:
# ru_model_name = "cointegrated/rubert-tiny2"
ru_model_name = "intfloat/multilingual-e5-large-instruct"
# ru_model_name = "sergeyzh/BERTA"
embeddings = get_embeddings(ru_model_name)

chroma_client_from_telegram = get_client(f'telegram_{CHANNEL_NAME}', embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

sentence_xlm-roberta_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [6]:
from langchain_chroma import Chroma
from datetime import datetime, timedelta, UTC

def get_last_seen_id_from_db(chroma_client: Chroma):
    doc_dict = chroma_client.get(
        where={"date": {"$gte": (datetime.now(UTC) - timedelta(days=30)).timestamp()}},
        include=['metadatas'],
    )
    if not doc_dict['metadatas']:
        return 0

    return max(meta['id'] for meta in doc_dict['metadatas'])

def get_earliest_message_id(chroma_client: Chroma):
    doc_dict = chroma_client.get(include=['metadatas'])
    if not doc_dict['metadatas']:
        return 0

    return min(meta['id'] for meta in doc_dict['metadatas'])

async def get_early_messages_from_chat(channel_id, telegram_client, chroma_client: Chroma, chunks=10):
    earliest_message_id = get_earliest_message_id(chroma_client)
    print("Earliest seen id:", earliest_message_id)
    messages = await get_messages_from_channel(
        telegram_client, channel_id, max_id=earliest_message_id, chunks=chunks
    )
    print("Got messages:", len(messages))
    return messages

async def get_last_messages_from_chat(channel_id, telegram_client, chroma_client: Chroma, chunks=10):
    last_seen_id = get_last_seen_id_from_db(chroma_client)
    print("Last seen id:", last_seen_id)
    messages = await get_messages_from_channel(telegram_client, channel_id, min_id=last_seen_id, chunks=chunks)
    print("Got messages:", len(messages))
    return messages

In [7]:
new_messages = await get_last_messages_from_chat(
    channel_id=CHANNEL_ID,
    telegram_client=telegram_client,
    chroma_client=chroma_client_from_telegram,
    chunks=10
)

Last seen id: 0


                                                                                                                                                                                           2.78s/it]

Got messages: 9282


In [30]:
new_messages = await get_early_messages_from_chat(
    channel_id=CHANNEL_ID,
    telegram_client=telegram_client,
    chroma_client=chroma_client_from_telegram,
    chunks=10
)

Earliest seen id: 80265


                                                                2.22s/it]

Got messages: 9141


In [8]:
print(len(new_messages))
print(list(new_messages.values())[-1:])

9282
[{'text': 'как ни странно, на пигу много чего дешевле, чем на амазоне даже с бесплатной доставкой', 'metadata': {'sender_name': 'sPRf77', 'id': 104554, 'date_str': '2024-04-19T10:07:19+00:00', 'date': 1713521239.0}}]


In [9]:
load_data_to_chroma(chroma_client_from_telegram, new_messages.values(), reset=True)

Loaded 9245 documents into ChromaDB.


In [11]:
from langchain import PromptTemplate
from langchain_ollama import ChatOllama


llm_qwen3_8b = ChatOllama(
    model="qwen3:8b",
)

In [46]:
user_query = "Выведи самые токсичные сообщения в чате."
user_query = "Выведи самые позитивные сообщения в чате."
user_query = "Выведи самые негативные сообщения в чате о Польше."
# user_query = "Можно ли компенсировать отель или аквапарк через велнес БТА/бта/BTA?"
# user_query = "Какие лучшие бани или сауны?"
# user_query = "Как получить детские деньги, компенсацию по уходу за ребенком."

In [19]:
ru_telegram_prompt_expanding = PromptTemplate(
    input_variables=["user_query", "n"],
    template="""
    Напиши {n} гипотетических ответов на запрос пользователя.
    Не добавляй заголовков, авторов и другой дополнительной информации к твоим ответам.

    Вопрос пользователя:
    {user_query}

    Гипотетические ответы:
    """,
)

hyde_chain_r1_8b = ru_telegram_prompt_expanding | llm_qwen3_8b
raw_hypothetical_answers = hyde_chain_r1_8b.invoke({"user_query": user_query, "n": 15}).content
raw_hypothetical_answers

NameError: name 'PromptTemplate' is not defined

In [25]:
user_query = raw_hypothetical_answers.split('</think>', maxsplit=1)[-1]

In [ ]:
Retrieve documents using embeddings multilingual-e5-large-instruct

In [45]:
query = f"Instruct: Given a question, retrieve relevant documents that best answer the question\nQuery: {user_query}"
related_docs = chroma_client_from_telegram.similarity_search_with_relevance_scores(query, k=15)

t_link_base = "https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D"
print("---".join(f"""
{score} {datetime.fromtimestamp(doc.metadata['date'], UTC)} {t_link_base}{doc.metadata['id']} - {doc.metadata['sender_name']}: {doc.page_content}
""" for doc, score in related_docs))


0.7988519106253629 2024-07-16 09:00:45+00:00 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D107464 - mnezaikin: >> в Польше не хотело работать

 В разных странах европы возникало. Такой вот роуминг)
---
0.7976308321303053 2024-07-16 08:53:23+00:00 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D107461 - trialerx: в Польше не хотело работать
---
0.7956835845088293 2025-04-08 16:01:22+00:00 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D113965 - Henkel1990: >> Ну значит дела не такие важные 🤷‍♂️
Сидеть боятся сьездить решить свои вопросы из-за аннулирования внж такое себе. Литва аннулирует, Польша выдаст, ну

 А Польша что всем и каждому ВНЖ выдает?
---
0.793342158486655 2025-04-08 16:00:13+00:00 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D113964 - dzyanislobka: >> Ну значит дела не такие важные 🤷‍♂️
Сидеть боят

In [16]:
related_docs = chroma_client_from_telegram.similarity_search_with_relevance_scores(f'search_query: {user_query}', k=15)
related_docs

[(Document(id='553187b7-3f91-4e4f-82b2-2d44ed8e5dc5', metadata={'date': 1741009166.0, 'id': 113241, 'sender_name': 'mnezaikin'}, page_content='>> А что по твоим критериям хороший и в каком районе\n\n район не сильно влияет, пушто я на машине\nХороший это "не подвал и с хорошей вентиляцией, с хорошими тренажерами, есть душ и есть шкафчики"'),
  0.8179634412294763),
 (Document(id='17c37982-9b66-4fdb-948d-316afa86eb43', metadata={'date': 1741016441.0, 'id': 113257, 'sender_name': 'mnezaikin'}, page_content='>> Фитус (новый) и форум (старый). Они с басиком и спа, это главный плюс. До фитуса ехать, до форума ножками пройтись от офиса. Форум подуставший уже, но вентиляция там норм и шаговая доступность. Фитус классный, дорого-богато и хорошо, но ехать в час пик туда обратно такое себе удовольствие(( По цене примерно одинаковые. Контингент приятный и там и там\n\n огонь, спасибо за фидбэк!'),
  0.8178154847177788),
 (Document(id='7d38b83e-7af7-4f4a-aa86-8aba8b65636e', metadata={'date': 174100

In [41]:

from datetime import datetime, timedelta, UTC

related_docs = chroma_client_from_telegram.get(
    where={"date": {"$gte": (datetime.now(UTC) - timedelta(days=30)).timestamp()}},
)
meta_docs = list(zip(related_docs['metadatas'], related_docs['documents']))

context = "\n\n---\n\n".join(f"{meta['date']} - {meta['sender_name']}: {doc}" for meta, doc in meta_docs)

In [42]:
len(related_docs['metadatas'])

778

In [43]:
filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.3, related_docs)

t_link_base = "https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D"

print("\n\n---\n\n".join(f"""
{doc.metadata['date']} {t_link_base}{doc.metadata['id']} - {doc.page_content}
""" for doc, _score in filtered_related_docs))

TypeError: '>' not supported between instances of 'str' and 'float'

In [ ]:
filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.3, related_docs)

context = "\n\n---\n\n".join(f"{doc.metadata['date']} - {doc.page_content}" for doc, _score in filtered_related_docs)

In [ ]:
# user_query = "Здесь собраны последние сообщения из чата. Расскажи про основные темы и выводы."

ru_telegram_prompt = PromptTemplate(
    input_variables=["context", "user_query"],
    template="""
    Ты полезный AI ассистент, который отвечает на вопросы пользователя на основе контекста.
    Контекст это релевантные вопросу сообщения из телеграм чата.

    Контекст:
    {context}

    Вопрос пользователя:
    {user_query}

    Если в контексте недостаточно информации, чтобы ответить на вопрос пользователя, то скажи, что недостаточно информации.

    Ответ:
    """,
)

hyde_chain_r1_8b = ru_telegram_prompt | llm_qwen3_8b
llm_response = hyde_chain_r1_8b.invoke({"user_query": user_query, "context": context}).content
print(llm_response)